In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: Friction

In [3]:
#http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=Friction&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=Friction

In [4]:
paper_link_list = []
#289
for i in range(289):
    before_url = "http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=Friction&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=Friction"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

Evaluation of Friction Properties According to Normal Force and Direction of Wood Grain in Real Contact Area
Engine Oil Jet System이 Engine Friction에 미치는 영향에 대한 연구
General Paper : Analyses of Influence of Frictional Heat on the Contact Stress of High-speed Micro-gears
Effects of Friction and Anvil Design on Plastic Deformation during the Compression Stage of High-Pressure Torsion
Research Papers : Elastomers and Composites ; A Study on Surface Treatment for Rubber Materials with Low Friction Factor
Impact of financial friction on productivity and international reserves with heterogeneous production functions
KJP : Case Report; Diagnosis of Iliotibial Band Friction Syndrome and Ultrasound Guided Steroid Injection
A Study on Adhesion Friction Characteristics of Rubber for Tire Tread
Mathematical Modeling of Friction Force in LM Ball Guides
Friction and Wear Characteristics of ABS-like Resin for 3D Printing Under Non-Lubricated Condition
10
Improvement of Abrasion Resistance and Friction o

사질토 지반에 설치된 현장타설말뚝의 말뚝지름에 따른 주면마찰력 분석
교통환기력이 작용하는 터널 내 벽면마찰계수 추정을 위한 프로그램 로직 개발
MR제어기의 마찰력비에 따른 단자유도 구조물의 응답감소
마찰 에너지 소산과 자동 복원력을 활용한 가새 댐퍼 시스템의 최적 설계와 구조적 활용
내부마찰각과 토압 산정방법에 따른 철도교대의 안정성 비교 연구
사질토층을 지나 풍화암에 소켓된 매입 PHC말뚝에서 지반의 허용압축지지력 산정도표 및 산정공식 개발에 관한 연구(I) - 재하시험 자료 분석을 통한 전체지지력에 대한 주면마찰력의 분담율(SRF) 분석 -
트로카 내부의 고무패킹에서 발생하는 마찰을 고려하여 수술 도구 말단에 작용하는 축 방향의 힘을 추정하는 방법
기구학적 운동이 인공관절 베어링용 초고분자량 폴리에틸렌 재료들의 마찰 특성에 미치는 영향 평가
가장자리 하중조건에서의 볼 조인트 마찰소음 연구
뒷채움재의 내부마찰각 변화에 따른 철도교대의 안정성 및 공사비 비교
220
Alloy718/SCM440 마찰용접재의 AE에 의한 동적 거동평가
스퀼 융합모델을 이용한 모드연성에 의한 빔 구조 마찰 소음 연구
시공줄눈이 있는 콘크리트 경계면의 전단마찰 내력에 대한 보강철근의 영향
햅틱 복강경 수술 시뮬레이터의 마찰력 및 중력 보상
Coupled Eulerian-Lagrangian기법을 이용한 이종 마찰교반용접 해석모델 개발
회전금형을 사용하는 AZ31 마그네슘 합금판재의전기저항 표면마찰 스폿용접
가스 스프링 높이 조절에 있어 마찰력 감소를 위한 피스톤 로드에 작용하는 가스씰 조임 기술 개발
양방향 재하시험 결과를 이용한 암반소켓 현장타설말뚝의 주면 마찰력과 선단 지지력
소성 이론에 의한 강섬유 보강 초고성능콘크리트의 전단 마찰 강도식 제안
알루미늄 합금 (Al6005-T6)의 마찰교반접합 시 공구의 회전속도와 접합 특성의 상관관계 연구
230
소성 이론에 의한 강섬유 보강 초고성능콘크리트의 전단 마찰 강도식 제안
가스 스프링 Elevation 동작 마찰력 보상 변수 조합 연

알루미늄합금과 타이타늄합금 판재에 대한 마찰교반용접 계면부의 미세조직
Hybrid 마찰교반접합기술에 관한 최근 연구
마찰교반접합기술을 이용한 이종재료 접합에 관한 최근 연구동향
마찰교반 점용접(FSSW)을 이용한 비철금속 접합기술의 연구 동향과 전망
마찰교반접합기술을 이용한 이종재료 접합에 관한 최근 연구동향
1928년 전라남도 도평의회의 조선인과 일본인 「알력」 사건 연구
고온재료의 마찰교반접합을 위한 툴의 개발
음향방출 특성을 이용한 마찰교반점용접 모니터링 기법 개발 동향
마찰교반접합부의 미세조직 관찰 및 유동해석
고온재료의 마찰교반접합을 위한 툴의 개발
430
SnO<sub>2</sub> 마찰층을 이용한 마찰 대전 소자의 에너지 생산성 향상
마찰교반용접 전산모사를 이용한 강도, 집합조직변화 고찰
음향방출 특성을 이용한 마찰교반점용접 모니터링 기법 개발 동향
마찰교반용접 전산모사를 이용한 강도, 집합조직변화 고찰
음향방출 특성을 이용한 알루미늄 합금의 마찰교반점용접 모니터링 기법 개발에 관한 연구
초음파 진동 딥 드로잉 공정에서의 마찰감소효과 분석을 위한 유한요소해석 및 실험
마찰에 의한 통증 입력이 작업자의 상지 고유수용성 감각에 미치는 영향
유엔사의 과거와 미래의 충돌 : 평화공헌론과 주권위축론의 마찰과 윈윈(win-win)전략 모색
라만 분광법을 활용한 마모 중 DLC 코팅의 구조적 변화 조사
마그네슘 및 알루미늄 합금의 마찰교반접합시 액화균열의 발생
440
환경잡음신호의 주파수특성 분석에 의한 전자보안펜스의 신뢰성 향상
지반의 전단변형에 따른 마찰형 앵커의 긴장력 변화에 대한 연구
도어트림 그립핸들 마찰이음 근본개선을 위한 내구 시험법 개발 및 개선연구
시장충격을 고려한 재무적 제약 기업의 현금보유와 연구개발 지속성
하이브리드 마찰교반접합된 니켈기 초합금의 미세조직과 기계적 특성 발달
하이브리드 마찰교반접합된 니켈기 초합금의 미세조직과 기계적 특성 발달
포장노면과 타이어간의 마찰음 분석을 통한 교통소음예측 소프트웨어 개발
급속소결 방법을 이용한 마찰

임의 최소주응력 구간에서 일반화된 Hoek-Brown 파괴기준식을 최적 근사하는 등가 Mohr-Coulomb 강도정수 계산식
다중이동로봇의 동적 모델링 및 구동성능 분석을 통한 새로운 바퀴 배치 제안
김일성의 6·25전쟁 전략 분석 : 전략의 전술화
김일성의 6.25전쟁 전략 분석
클러치 드래그 토크에 미치는 마찰재 면적 및 클리어런스의 영향
공기윤활선 모사 실험에서의 공극률 및 마찰저항저감율 상관성 분석을 위한 실험적 연구
관성에 따른 소결마찰재와 제동디스크간 마찰특성 연구
일반국도에 적용한 마이크로서페이싱공법과 폴리머슬러리실공법에 대한 현장 공용성 평가
미세 딤플의 밀도에 따른 SCM415강의 마찰 거동 연구
일반국도에 적용한 마이크로서페이싱공법과 폴리머슬러리실공법에 대한 현장 공용성 평가
660
침자 술기 실습 모델에서 침감의 정량적 특성에 관한 연구
간선도로 좌곡선부 전후구간 수막현상 방지를 위한 종.횡단경사 조합 적용방안
간선도로 좌곡선부 전후구간 수막현상 방지를 위한 종 횡단경사 조합 적용방안
사질토를 지나 풍화암에 소켓된 매입 PHC말뚝에서 지반의 허용압축지지력 산정도표 및 산정공식 개발에 관한 연속 연구(V) - 매개변수 수치해석 자료 분석 -
열화 및 교통하중에 의한 소입경 골재노출 콘크리트 포장의 마모 및 탈리 특성
상품의 브랜딩을 위한 차별화된 휴먼 인터페이스 모델 개발
백토안료의 전통제법 중 아교수 영향에 관한 연구
SiO<sub>2</sub> 나노 콜로이드 량이 다른 Si<sub>3</sub>N<sub>4</sub>의 열처리에 따른 마모 특성
격자, 난류모형 및 이산화 방법이 유동해석 결과에 미치는 영향
해저터널 급속차폐를 위한 팽창구조체의 설계 및 현장적용에 대한 연구
670
Effect of Water Content on Tribological Characteristics of Grease
세가지 마찰저항곡선의 특성에 관한 연구
상대재료의 표면거칠기에 다른 PTFE와 UHMWPE의 마찰 및 마멸 특성
연구논문 : 칼날형 마모시험기를 

논문 : 북평분지와 포항분지 시추코어의 물리적 성질과 역학적 성질간의 관계
경제자유화 정도에 따른 내재적 조세의 실현가능성: Heritage Foundation의 경제자유화지수를 중심으로
공정 온도에 따른 사면체 비정질 카본 (ta-C) 코팅의트라이볼로지적 특성연구
이중 슬립마찰면을 이용한 면진장치의 면진성능평가
만곡부에서 이론식에 기반한 횡분산계수 경험공식 개발
에어컨 실내기의 수축팽창 소음 저감 방법
틸팅차량용 휠 제동장치의 스퀼 소음 해석
습식 브레이크의 소음 저감에 대한 연구
디스크 브레이크의 구조 및 열 해석
디스크 브레이크의 구조 및 열 해석
890
일반화된 Hoek-Brown 암반의 등가 Mohr-Coulomb 강도정수 산정법 고찰
호안 경사와 인공사석의 배열이 파랑에너지에 미치는 영향 분석
해석적 접근을 통한 기어 마찰 동력 손실 예측 및 최소화
타이어 뉴메틱 트레일 정보를 활용한 횡방향 타이어 노면 마찰 계수에 관한 연구
사막먼지 발생량 산정 모델 개발 현황
차세대 고속철 주행속도를 대비한 교량받침의 장기마찰시험법
이상 유동에 놓인 관군의 표면에 작용하는 압력 분포
스크린 프린팅 기술을 적용한 피스톤 스커트의 브레이드 코팅공정에 관한 기초연구 - 사례연구
타설 경계면을 고려한 슬라이딩 궤도 횡방향 지지 콘크리트 블록의 전단 내하력 평가
수평 사각 채널에서의 2상 압력 강하
900
Laser Texturing한 평행 스러스트 베어링의 윤활특성
반작용휠 저속구간에서의 위성자세제어
중일 도서 영유권 분쟁 원인에 대한 중국 국내 전문가 인식조사
중첩된 구리 판재의 전기저항가열 표면마찰 점용접(RSFSW)에 관한 연구
[Review] 트라이볼로지 관점에서의 그래핀 분자시뮬레이션 연구동향
SLM방식으로 출력된 STS 316L의 기계적 및 마찰 · 마모특성에 미치는 UNSM처리 후 영향에 관한 연구
금융중개부문 신용마찰과 경기변동
곡관부 열전달 성능 강화를 위한 에어포일형 가이드 베인의 형상 최적설계
중일 도서 영유권 분쟁 원인에 대한 중국 국내 전문가 인

...

Large-Eddy Breakup Device가 수중운동체의 저항에 미치는 영향
나노 채널에서의 표면 거칠기와 경계 습윤의 효과
원자력발전 설비의 소재와 용접방법에 대한 최신 기술동향
한국의 산업 미스매치 실업
리드프레임의 전단용 금형에 대한 3차원 FEM 해석
고속카메라 데이터 분석을 통한 발사체 지지대 분산 궤적의근사적 예측 방법
연마가공 노즐의 성능개선을 위한 실험적 연구
기계 가공공정 유한요소법 해석 기술의 현황
고려 靖宗代 入仕와 직역 승계 규정
일본 경단련의 정치적 연관성에 관한 연구
1770
DLC 코팅된 SACM645 소재의 마모 특성
나노모터의 기가급 공진 특성에 대한 연구
4절링크 기구기반의 회전형 초정밀위치결정기구의 개발
경사방향 추정 기법을 이용한 소형로봇의 퍼지 조향 제어
저 T/T<SUB>m</SUB> 온도에서 공석강 및 과공석강의 시간의존성 소성변형 기구
충격파 터널에서의 가속도계 기반 항력 측정
中部地方 粘土帶土器 段階 聚落 構造와 性格
Comparison of the physical characteristics according to the varieties of perilla for the development of a high-quality, high-efficiency cleaner and stone separator
클라우제비츠 삼위일체의 현대적 해석
중등과학 실험을 활용한 관성질량 측정에 대한 연구
1780
폴리에틸렌이민 전처리와 황화구리 무전해 도금에 의한 전기전도성 면섬유의 제조
저 T/Tm 온도에서 공석강 및 과공석강의 시간의존성 소성변형 기구
직접 외란 추정을 통한 역구동성 유압 구동 시스템의 임피던스 제어
수치해석을 통한 이중 베이스플레이트 연결부의 리브 구조 상세에 대한 연구
건조기용 타원관 대구경 핀-관 열교환기의 성능특성
석재 표면가공을 위한 자동 고속버너 가공기 개발
중국의 청산방해죄 성립요건에 대한 법리해석과 판례분석
원심모형실험을 이용한 모래지반에 관입된 계류선 거동 평가
회전관절만을 활용하는 병진 3

KTX 제동장치의 고유진동수와 스퀼소음 분석
분말붕소법을 이용한 오스테나이트계 스테인리스 316L강의 기계적 특성 향상을 위한 연구
포장노면 미끄럼 저항특성
Point-Collocation Non-intrusive Polynomial Chaos 기법을 활용한 평판 주위 난류 유동장 해석
200 마력급 터보 블로워 적용을 위한 자기베어링 설계
정밀가공을 위한 CNC 알고리즘 연구 현황
준설토 이송시 유동효율에 미치는 전자기장 인가 영향에 대한 실험적 고찰
FIB를 이용한 트라이보층에 대한 연구
변위 제어형 자동 약물주입기의 구동기구 동역학 해석
C-MDPS 차량의 응답 성능 최적화 연구
1990
차량 연비개선을 위한 자동변속기유 열교환기에 대한 실험적 연구
훈민정음의 원형 자음과 원형 모음의 과학적 원리
자기부상 물류이송시스템 설계에 관한 연구
캐스터블 내화물에 충돌하는 입자의 입사각도가 에로젼에 미치는 영향
절삭유 분사위치에 따른 STS316L의 밀링가공 특성 개선
레이저 가공에서 동축/탈축 보조가스의 충돌특성에 관한 수치해석적 연구
Design and Evaluation of the Control Performance of a Compliant Arm Support
자동차 내장재용 고분자 재료의 필드 열화에 따른 마찰소음 특성변화
베인형 진공펌프의 내부유동과 구조 강성에 관한 해석적 연구
외바퀴 로봇 다이나믹 모델과 성능 개선
2000
타워 관측 자료를 이용한 연안 대기 경계층 내 바람 자원의 연직 변동 특성
고층 공동주택의 승강로가압을 이용한 차압 및 방연풍속에 관한 연구
한국국제협력단(KOICA)의 태권도교육사업에서 나타난 문제들
문헌조사에 근거한 부직포 보강토옹벽의 거동에 관한 연구
무수말레인산이 그라프트된 폴리프로필렌 분말 첨가에 따른 시멘트 모르타르와 무극성 마크로 합성섬유의 부착 특성
차량 부품의 노면 가진 특성을 고려한 래틀과 스퀵 현상 검출 방법의 개발
터보챠저 저어널 베어링에서 물과 윤활유가 혼합될 때 베어링 성능에 관한 연구
‘청년' 연애학

시험평가법을 이용한 IRB 면진장치 롤러 설계
사판식 유압 피스톤 펌프의 응력해석에 관한 연구
음향방출법을 이용한 CFRP 복합재의 모드 1 : 파괴거동에 관한 연구
투자효율성이 회계이익의 지속성과 가치관련성에 미치는 영향
줄리아 언어를 이용한 고성능 해양모델의 개발
테프론 또는 제강슬래그를 활용한 기초형 지진격리장치의 면진 메카니즘 평가
핀란드 초등 과학 교과서의 소재중심 통합단원 분석
보행 보조용 고관절 외골격 로봇의 Negative Damping 제어 기법
복합 전열 촉진 핀이 적용된 핀-관 열교환기의 성능에 대한 실험적 연구
초고속화염용사 WC-CoFe 코팅층의 레이저 표면 열처리 효과
2210
다양한 그루브 단면형상에 대한 스풀밸브의 윤활특성 연구
흑연소재가공용 공구의 DLC 코팅두께가 가공특성에 미치는 영향
서해안 저소성 점토질 실트 지반의 부분배수 특성
신라 중고기 왕실여성과 불교 ― 영흥사의 창건과 도유나랑을 중심으로 ―
장경간 철도 교량에 적용된 슬라이딩 궤도와 레일신축이음장치의 궤도-교량 상호작용 비교
CFD를 이용한 나선형 튜브 열교환기에서 강제대류의 열유동특성에 관한 연구
Performance Test of 3D Printed Blades for a Scaled Wind Turbine in a Wind Tunnel
개인의 입신, 근대교육 그리고 국가: 『무정』과 『도련님』을 통해 본 사회적 관계의 재편
생체모사 소프트 그리퍼 설계 및 제작
물리적 불확실성을 내재한 입력변수의 확률 통계 기반 유효 범위 결정 방법 및 신뢰성 평가
2220
영어마찰음의 지각단서와 동화작용
선박 추진용 유압작동식 다판 마찰클러치 조립축 크랙 현상 고장탐구
쉐브론 형상 판형 열교환기의 고온 채널에서의 압력손실 및 열전달 특성에 관한 해석 연구
분포정수계 유압관로 모델의 동특성 해석
판형 열교환기에서 에틸렐글리콜 수용액의 열전달 및 압력강하 특성에 관한 실험적 연구
산업부산물을 이용한 비소성 고화제 혼합토의 역학적 특성
호찌민 원칙중심의 실용주의 외교전략 고찰 - 

구동기 동역학을 가지는 이동 로봇에 대한 FBFN을 이용한 강인 적응 퍼지 추종 제어
행동과학에 기초한 주요국의 납세자 친화적 세무행정 및 정책시사점
화물차 주위 유동의 성긴 격자 큰에디모사
재난약자로서 시각장애인의 코로나19 경험에 관한 연구
보강토 지반에서 환경보전 및 토공저감을 위한 앵커판의 인발저항효과
유한요소해석과 다구찌 방법을 이용한 클린치 스터드의 설계 최적화
Evaluation of Driving Performance of Wheel Loader in the Framework of Fitts' Law
스크래치 방지를 위한 테이퍼 롤링형 판재 누름키의 개발
선박발전기용 디젤엔진의 부분부하에서 에너지 효율 개선에 관한연구
유압동력 발생장치의 소음특성 개선을 위한 실험적 연구
2440
유리 섬유복합체(GFRP) 전단연결재로 보강된 중단열 콘크리트 월 패널의 단열재 종류에 따른 휨 거동 분석
사질토지반의 지지력분석을 위한 얕은기초의 파괴거동에 대한 모형실험과 유한요소해석 비교 검토
SHALSTAB을 이용한 산사태 위험지 예측
플래퍼론이 전개된 플라잉윙 형상의 공력 특성에 대한 전산유동해석
준설토 장거리 이송기술 분석 및 평가
볼텍스 세퍼레이터(Vortex Separator) 내부 입자 유동 특성 해석
장기간 사용한 이중관 의치의 실패원인 및 자연치와 임플란트를 이용한 이중관 의치 수복증례
열수변질 점토맥이 사면 안정성에 미치는 영향에 관한 모델링 연구
나노다이아몬드 첨가 윤활제가 디젤엔진의 성능에 미치는 영향
자동차 조향장치의 트라이볼로지적 고장특성에 관한 사례연구
2450
사질토지반의 선단확장형말뚝의 인발거동 특성
돌기 접촉 모델과 평균 유동 분석을 이용한 딤플 패턴의 윤활 특성에 관한 연구
차륜-레일의 동적효과를 고려한 측풍 원인 탈선 예측방법 연구
위치인식기반서비스를 활용한 능동형 도로기상 교통정보시스템
고려 인종대 對여진(금) 관계 설정과 현실주의적 대응
거래소 간 자산 교환비율 차이를 이용한 차익거래기회 분석
도로터널 화재시 열부력이 제연용 제

리브 간격 변화에 따른 열ㆍ유동 수치해석 및 압력 저하 특성
고체산화물 연료전지 단위셀의 열응력에 관한 연구
그리스 코린트 만(灣)의 1995년 에기오(Aigio) 지진과 지진 전조현상- 가스 방출 메커니즘 -
3차원 유한요소해석에 의한 얕은 기초의 지지력 특성
주택재개발사업에서의 주체별 개발이익의 추정
21세기 한국교회 교육목회의 위기 분석 및 대안 제시 -장년교육목회를 중심으로-
김대중, 노무현 정부의 대북정책과 국내정치: 문제는“밖”이 아니라“안”이다
가지를 면과 대나무 섬유에 적용한 천연염색 고찰
공기 온습도가 공조용 가습 소자의 가습 성능에 미치는 영향에 대한 실험 연구
철재형 이안제의 설계외력 산정기법 검증을 위한 실해역 시험
2660
네덜란드·러시아 간 Arctic Sunrise호 사건의 분석 및 시사점:임검권과 추적권을 중심으로
폴리올레핀계 합성 섬유와 시멘트 모르타르와의 부착 특성에 미치는 광물질 혼화재의 효과
대학생들의 다양한 생리용품 사용 경험
무수말레인산이 그라프트된 폴리프로필렌 분말 첨가에 따른 시멘트 모르타르와 무극성 마크로 합성섬유의 부착 특성
사면 및 터널에서의 암반 파쇄대 보강을 위한 개량형 록볼트 개발
토지제도의 공법적 검토-토지공개념 관련 법제를 중점으로-
강우에 의한 암반사면 파괴 해석 사례 연구를 통한 해석방법 적용성 검토
ESD 접지선 단선 모니터링 시스템 설계 및 구현
Open face 터널시공으로 인한 단독말뚝의 거동
도시철도 급곡선구간에서 레일 파상마모가 윤중 및 횡압에 미치는 영향
2670
삼축압축시험에 의한 국내 모래의 변형-강도 특성
미국의 동북아시아 냉전전략과 샌프란시스코 체제의 형성: 전후 영토문제와 배상문제를 중심으로
자동차 부품의 표면처리에 사용되는 가스 침탄질화처리 대체기술로 QT (퀜칭과 템퍼링)와 결합된 플라즈마 침질탄화 기술개발
해성점토의 지반 강도정수 상관성 분석
한국 ‘사회혁신'의 지형도: 새로운 통치합리성과 거버넌스 공간의 등장
선박 추진축계 선미관 베어링의 강제 압입 피팅 방식에 관한 연구

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d3360632359243b5ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e7be87b6819f0f5ce9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=755f4e142adb7f9fb36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=cbbe7e0fa5d672577ecd42904f0c5d65',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7bd1401e1d8c930ce9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d2d8dd0ceb068468d18150b21a227875',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=087b1f99119e5debffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=23c2edafec1a090e6aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=22d557718b78c21ab7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

2883

In [7]:
paper_url_1 = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url_1.append(tmp_url)

100%|█████████████████████████████████████████████████████████████████████████| 2883/2883 [00:00<00:00, 1400368.09it/s]


In [8]:
paper_url_1

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d3360632359243b5ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e7be87b6819f0f5ce9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=755f4e142adb7f9fb36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=cbbe7e0fa5d672577ecd42904f0c5d65',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7bd1401e1d8c930ce9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d2d8dd0ceb068468d18150b21a227875',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=087b1f99119e5debffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=23c2edafec1a090e6aae8a972f9116fb',
 'http://www.ris

In [9]:
len(paper_url_1)

2883

# 검색키워드: 마찰력

In [10]:
#http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%A7%88%EC%B0%B0%EB%A0%A5&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%A7%88%EC%B0%B0%EB%A0%A5

In [11]:
paper_link_list = []
#29
for i in range(29):
    before_url = "http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%A7%88%EC%B0%B0%EB%A0%A5&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%A7%88%EC%B0%B0%EB%A0%A5"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

일반물리학 교재에 나타난 마찰력 관련 서술의 특징: 요철설과 응착설을 중심으로
변위 의존 마찰력 방진장치의 진동저감 특성
교정용 브라켓의 종류와 각도, 호선의 코팅 여부에 따른 마찰력의 비교
주간보고서에 나타난 전공역학 수강생들의 마찰력에 대한 어려움과 그 어려움의 원인 구조
마찰력과 관련된 기본 역학개념 형성에 관한 연구
LM 볼가이드의 마찰력 정식화
현장시험을 통한 기초 말뚝 부마찰력의 특성과 시공관리
중·고등학생의 마찰력 개념 이해의 어려움 분석과 한 가지 지도전략의 제안
구르는 물체에 작용하는 마찰력에 대한 과학 영재학교 학생들의 이해의 특징
고무동력바퀴를 활용한 마찰력 수업에 관한 연구
10
원전 동력구동 밸브 패킹 마찰력 예측에 관한 연구
물리교육을 위한 운동마찰력 측정 방법의 제안
가스 스프링 Elevation 동작 마찰력 보상 연구 - 가스 씰 조임 기술 연구 및 피스톤 로드의 부피 보상 장치
공통맥락 형성의 관점에서 살펴본 마찰력에 대한 소집단 토론의 특징
MR제어기의 마찰력비에 따른 단자유도 구조물의 응답감소
가스 스프링 Elevation 동작 마찰력 보상 연구 - 피스톤 로드의 부피 보상 장치 및최적화된 피스톤 구조 개발
양방향 재하시험 결과를 이용한 암반소켓 현장타설말뚝의 주면 마찰력과 선단 지지력
마찰력과 강성을 고려한 사출성형기 5절 토글 링크에 관한 연구
가스 스프링 Elevation 동작 마찰력 보상 연구(1) - 관형 오리피스 단면 및 가스씰 조임 기술 개발
가스 스프링 Elevation 동작 마찰력 보상 연구 - 가스 씰 조임 기술 연구 및 최적화된 피스톤 구조 개발
20
적응화 평활화법을 이용한 다기능 마찰력 측정기의 성능 분석
모터 제어 정밀도 향상을 위한 정지 마찰력 보상
가스 스프링 높이 조절에 있어 마찰력 감소를 위한 피스톤 로드에 작용하는 가스씰 조임 기술 개발
가스 스프링 Elevation 동작 마찰력 보상 변수 조합 연구 (1)
햅틱 복강경 수술 시뮬레이터의 마찰력 및 중력 보상
마찰력을 고려한 군용 MR 현수 장

무리말뚝을 구성하는 개별말뚝의 선단지지력에 대한 실험연구
사질토 지반에서 N값과 말뚝의 길이비가 지지력 분담 특성에 미치는 영향
환산SPT N값을 이용한 현장타설말뚝의 단위주면마찰지지력 산정
사질토를 지나 풍화암에 소켓된 매입 PHC말뚝에서 지반의 허용압축지지력 산정도표 및 산정공식 개발에 관한 연구(VI) - 지반의 허용압축지지력 산정용 표해 또는 도해 -
사질토 지반에 설치된 버킷기초의 수직 하중전이 특성
흐름저항응력 및 초기수심에 따른 댐붕괴류의 수리특성
동결된 화강풍화토와 알루미늄판 접촉면에서 발현되는 동착강도 측정 연구
확공형 마이크로 파일의 지지력 해석
직경 600mm PHC 매입말뚝의 동재하시험을 통한 허용 지지력 평가
사질토층을 지나 풍화암에 소켓된 매입 PHC말뚝에서 지반의 허용압축지지력 산정도표 및 산정공식 개발에 관한 연구(II) - 설계 사례 분석을 통한 매입 PHC말뚝의 설계 개선 방향 -
240
일반화된 Hoek-Brown 파괴조건식에 내포된 접선점착력과 접선마찰각의 상관성
관입형 프로브의 온도전이 특성
지반과 쏘일네일링 사이의 전단거동에 관한 연구
강관 매입말뚝의 주면 하중전이 곡선(t-z) 제안
쏘일네일링과 앵커가 결합된 하이브리드 공법의 하중전이 메커니즘
쐐기수평력을 도입한 무그라우팅 선단압축 마이크로파일의 지지력 특성에 관한 연구
24주간 복합 트레이닝이 고령자들의 근력, 에너지기질과 염증인자에 미치는 영향
재하시험을 통한 초고강도 선단확장 PHC말뚝의 적용성 연구
초기항타 및 재항타 동재하시험 결과를 조합한 매입말뚝의 하중-침하량 곡선 산정
RC 슬래브와 SC 벽 접합부의 전단마찰 거동에 관한실험연구
250
산화 텅스텐 나노막대의 트라이볼로지 특성
뒷굽이 있는 케이슨 안벽에 작용하는 토압에 대한 연구
세포군집의 확장에 관여하는 물리적 힘의 가시화
동토 플랜트 유체기계 구조물 설치를 위한 PET 골재적용 말뚝의 주면작용 수평력 평가
가상현실에서 물리적 현상들과 공간관계들의 표현
소형 콘의 온도보상 기법 연구
진동해머에 의해 시공

In [12]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=4500f467aa1c1c09b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=70bbf96792e479aa47de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=295d266edaa64a90ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7d182bbdc4aa194c6aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ff994f2fd608ebb06aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=63f2b3c143796100b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=36a98bd0e4f809a56aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a6206e5d4ad003ad47de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0d6ee0438b3264bd6aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [13]:
len(paper_link_list)

283

In [14]:
paper_url_2 = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url_2.append(tmp_url)

100%|████████████████████████████████████████████████████████████████████████████████████████| 283/283 [00:00<?, ?it/s]


In [15]:
paper_url_2

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=4500f467aa1c1c09b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=70bbf96792e479aa47de9c1710b0298d',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=295d266edaa64a90ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7d182bbdc4aa194c6aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ff994f2fd608ebb06aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=63f2b3c143796100b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=36a98bd0e4f809a56aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a6206e5d4ad003ad47de9c1710b0298d',
 'http://www.ris

In [16]:
len(paper_url_2)

283

In [17]:
paper_url = paper_url_1 + paper_url_2
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d3360632359243b5ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e7be87b6819f0f5ce9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=755f4e142adb7f9fb36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=cbbe7e0fa5d672577ecd42904f0c5d65',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7bd1401e1d8c930ce9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d2d8dd0ceb068468d18150b21a227875',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=087b1f99119e5debffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=23c2edafec1a090e6aae8a972f9116fb',
 'http://www.ris

In [18]:
len(paper_url)

3166

In [19]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [20]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [21]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [22]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./engin_friction.xlsx')
    
print("Running Time : ", time.time() - start)

0 paper clear
1 paper clear
2 paper clear
3 ERROR
4 paper clear
5 ERROR
6 ERROR
7 paper clear
8 paper clear
9 paper clear
10 ERROR
11 paper clear
12 paper clear
13 paper clear
14 paper clear
15 paper clear
16 paper clear
17 paper clear
18 ERROR
19 paper clear
20 ERROR
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 paper clear
30 paper clear
31 paper clear
32 paper clear
33 paper clear
34 paper clear
35 paper clear
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 paper clear
43 paper clear
44 paper clear
45 paper clear
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 ERROR
57 paper clear
58 paper clear
59 paper clear
60 paper clear
61 paper clear
62 paper clear
63 paper clear
64 paper clear
65 paper clear
66 paper clear
67 paper clear
68 paper clear
69 paper clear
70

555 paper clear
556 paper clear
557 paper clear
558 paper clear
559 paper clear
560 paper clear
561 paper clear
562 paper clear
563 paper clear
564 paper clear
565 paper clear
566 paper clear
567 paper clear
568 paper clear
569 paper clear
570 paper clear
571 paper clear
572 paper clear
573 paper clear
574 paper clear
575 paper clear
576 paper clear
577 paper clear
578 paper clear
579 paper clear
580 paper clear
581 paper clear
582 paper clear
583 paper clear
584 paper clear
585 paper clear
586 paper clear
587 paper clear
588 paper clear
589 ERROR
590 paper clear
591 paper clear
592 paper clear
593 paper clear
594 paper clear
595 paper clear
596 paper clear
597 paper clear
598 paper clear
599 paper clear
600 paper clear
601 paper clear
602 paper clear
603 paper clear
604 paper clear
605 paper clear
606 paper clear
607 paper clear
608 paper clear
609 paper clear
610 paper clear
611 paper clear
612 paper clear
613 paper clear
614 paper clear
615 paper clear
616 paper clear
617 paper clea

1072 paper clear
1073 paper clear
1074 paper clear
1075 paper clear
1076 paper clear
1077 paper clear
1078 paper clear
1079 paper clear
1080 paper clear
1081 paper clear
1082 paper clear
1083 paper clear
1084 paper clear
1085 paper clear
1086 paper clear
1087 paper clear
1088 paper clear
1089 paper clear
1090 paper clear
1091 paper clear
1092 paper clear
1093 paper clear
1094 paper clear
1095 paper clear
1096 paper clear
1097 paper clear
1098 paper clear
1099 paper clear
1100 paper clear
1101 paper clear
1102 ERROR
1103 paper clear
1104 paper clear
1105 paper clear
1106 paper clear
1107 paper clear
1108 paper clear
1109 ERROR
1110 paper clear
1111 paper clear
1112 paper clear
1113 paper clear
1114 paper clear
1115 ERROR
1116 paper clear
1117 paper clear
1118 paper clear
1119 paper clear
1120 paper clear
1121 paper clear
1122 paper clear
1123 paper clear
1124 paper clear
1125 ERROR
1126 paper clear
1127 paper clear
1128 paper clear
1129 paper clear
1130 paper clear
1131 paper clear
1132

1571 paper clear
1572 paper clear
1573 paper clear
1574 ERROR
1575 paper clear
1576 paper clear
1577 ERROR
1578 ERROR
1579 paper clear
1580 ERROR
1581 paper clear
1582 paper clear
1583 paper clear
1584 paper clear
1585 paper clear
1586 paper clear
1587 paper clear
1588 paper clear
1589 paper clear
1590 paper clear
1591 paper clear
1592 paper clear
1593 ERROR
1594 paper clear
1595 paper clear
1596 paper clear
1597 paper clear
1598 paper clear
1599 paper clear
1600 paper clear
1601 paper clear
1602 paper clear
1603 paper clear
1604 paper clear
1605 paper clear
1606 paper clear
1607 paper clear
1608 paper clear
1609 paper clear
1610 paper clear
1611 paper clear
1612 paper clear
1613 paper clear
1614 paper clear
1615 paper clear
1616 paper clear
1617 paper clear
1618 paper clear
1619 paper clear
1620 paper clear
1621 paper clear
1622 paper clear
1623 paper clear
1624 paper clear
1625 paper clear
1626 paper clear
1627 paper clear
1628 paper clear
1629 paper clear
1630 paper clear
1631 paper

2071 paper clear
2072 paper clear
2073 paper clear
2074 paper clear
2075 paper clear
2076 ERROR
2077 paper clear
2078 paper clear
2079 paper clear
2080 paper clear
2081 paper clear
2082 paper clear
2083 paper clear
2084 paper clear
2085 ERROR
2086 paper clear
2087 paper clear
2088 paper clear
2089 paper clear
2090 paper clear
2091 paper clear
2092 paper clear
2093 paper clear
2094 ERROR
2095 paper clear
2096 paper clear
2097 paper clear
2098 paper clear
2099 paper clear
2100 paper clear
2101 paper clear
2102 paper clear
2103 paper clear
2104 paper clear
2105 paper clear
2106 paper clear
2107 paper clear
2108 ERROR
2109 paper clear
2110 paper clear
2111 paper clear
2112 paper clear
2113 paper clear
2114 paper clear
2115 paper clear
2116 paper clear
2117 paper clear
2118 paper clear
2119 paper clear
2120 ERROR
2121 paper clear
2122 paper clear
2123 paper clear
2124 paper clear
2125 paper clear
2126 paper clear
2127 paper clear
2128 ERROR
2129 paper clear
2130 paper clear
2131 paper clear

2566 paper clear
2567 paper clear
2568 paper clear
2569 paper clear
2570 paper clear
2571 paper clear
2572 paper clear
2573 paper clear
2574 paper clear
2575 paper clear
2576 paper clear
2577 paper clear
2578 paper clear
2579 paper clear
2580 paper clear
2581 paper clear
2582 paper clear
2583 paper clear
2584 paper clear
2585 paper clear
2586 paper clear
2587 paper clear
2588 paper clear
2589 paper clear
2590 paper clear
2591 paper clear
2592 paper clear
2593 paper clear
2594 paper clear
2595 paper clear
2596 paper clear
2597 paper clear
2598 paper clear
2599 paper clear
2600 paper clear
2601 paper clear
2602 paper clear
2603 paper clear
2604 paper clear
2605 paper clear
2606 paper clear
2607 paper clear
2608 paper clear
2609 paper clear
2610 paper clear
2611 paper clear
2612 paper clear
2613 paper clear
2614 paper clear
2615 paper clear
2616 paper clear
2617 paper clear
2618 paper clear
2619 paper clear
2620 paper clear
2621 paper clear
2622 paper clear
2623 paper clear
2624 paper cle

3051 paper clear
3052 paper clear
3053 paper clear
3054 paper clear
3055 paper clear
3056 paper clear
3057 paper clear
3058 paper clear
3059 paper clear
3060 paper clear
3061 paper clear
3062 paper clear
3063 paper clear
3064 paper clear
3065 paper clear
3066 paper clear
3067 paper clear
3068 paper clear
3069 paper clear
3070 paper clear
3071 paper clear
3072 paper clear
3073 paper clear
3074 paper clear
3075 paper clear
3076 paper clear
3077 paper clear
3078 paper clear
3079 paper clear
3080 paper clear
3081 paper clear
3082 paper clear
3083 paper clear
3084 paper clear
3085 paper clear
3086 paper clear
3087 paper clear
3088 paper clear
3089 paper clear
3090 paper clear
3091 paper clear
3092 paper clear
3093 paper clear
3094 paper clear
3095 paper clear
3096 paper clear
3097 paper clear
3098 paper clear
3099 paper clear
3100 paper clear
3101 paper clear
3102 paper clear
3103 paper clear
3104 paper clear
3105 paper clear
3106 paper clear
3107 paper clear
3108 paper clear
3109 paper cle